<a href="https://colab.research.google.com/github/Rohitv97/Data-Science-CE888/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# **Jester Dataset**
1. This is only part of the entire dataset. This contains 24983 rows and 101 columns.
2. Each row corresponds to a user.
3. First column is number of jokes rated by that user
4. Others are ratings for each joke. Ratings are in the range of -10 to 10. Value of 99 indicates Null. That is no rating.

All information was gained by looking at the dataset and from the webpage of the [dataset](http://eigentaste.berkeley.edu/dataset/). 


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [3]:
## Copy values and keep
test_data = df.copy()
df = df.loc[:, 1:]  ##first column is just number of jokes, and not ratings
arr = df.values
print(arr.shape)

(24983, 100)


In [4]:
## Take all rated jokes. If not rated don't add.
## np.where returns an ndarray with all elements which satisfy the condition (x), and then all elements which don't satisfy the condition
## Over here first array is all rows, and 2nd array is all columns indices where it is not 99
## Use this to understand, why it is used inside the Replace Function
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [88]:
print(idx)
print(idx[0][1], idx[1][1])

(array([24320, 24804, 20872, ..., 20468, 21321, 24034]), array([69, 58, 89, ..., 30, 20, 25]))
24804 58


# **Latent Factor**

# Notes on training

1. It takes approx 36 seconds to train 1 iteration. 
300,000 iterations is possible only if enough computation power is available.

2. As of now, for normal training, after 9th iteration, the MSE starts increasing again. This could be due to overfitting. Try modifying hyperparameters to find convergence and stopping criteria.

# Training results

Alpha = 0.0001 -> around 9 iterations. (lowest MSE = 426.) Increases after this.

Alpha = 0.001 & Decay = 0.1 -> around 7 iterations (lowest MSE = 402.) Starts increasing after this value.

Alpha = 0.001 & Regularization = 0.1  and Decay = 0.1-> around 7 iterations (lowest MSE = 404). Increases after this.

Alpha = 0.001 & Regularization = 0.1  and Decay = 0.1. Also using more latent features (10). It was 4 before this. -> around 12 iterations. (lowest MSE = 366). Increases after this.

Alpha = 0.001 & Regularization = 0.1  and Decay = 0.1. 20 LF -> around 15 iterations. (lowest MSE = 299). Increases after this.

**This means we can use more Latent Features. Theoretically, more LFs should result in a lower MSE, with the trade off being computation power and time.**

In [0]:
n_latent_factors = 50

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001, weight_decay=0.1):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values)
    return err
    

def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                # if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 100) == 0:
        #     print('MSE ', iteration, ': ', mse)
        print('MSE ', iteration, ': ', mse)


In [38]:
sgd()

MSE  0 :  28.371271027917963
MSE  1 :  25.015122496766043
MSE  2 :  24.928617926547922
MSE  3 :  24.859419332998346
MSE  4 :  24.799191005829137
MSE  5 :  24.740883428413046
MSE  6 :  24.677474671788037
MSE  7 :  24.600821765960504
MSE  8 :  24.500482956117423
MSE  9 :  24.36252042159121
MSE  10 :  24.168562271179628
MSE  11 :  23.895898066889128
MSE  12 :  23.520090330319395
MSE  13 :  23.021998504031398
MSE  14 :  22.39974911161156
MSE  15 :  21.681305682977587
MSE  16 :  20.926911020168166
MSE  17 :  20.212212621699134
MSE  18 :  19.59878373609533
MSE  19 :  19.113551662074634
MSE  20 :  18.750327093312162
MSE  21 :  18.48542212803414
MSE  22 :  18.292347350029992
MSE  23 :  18.149074649333958
MSE  24 :  18.03950309751691
MSE  25 :  17.952486019377837
MSE  26 :  17.880421341137655
MSE  27 :  17.81809748266523
MSE  28 :  17.761888199467926
MSE  29 :  17.70922128585818
MSE  30 :  17.658233519625828
MSE  31 :  17.607546922238065
MSE  32 :  17.556123563293
MSE  33 :  17.50317159041398
M

In [0]:
def validation_check():
  error = []
  prediction = latent_user_preferences.dot(latent_item_features.T)
  for i in range(len(idx[0])):
    predicted_rating = prediction[idx[0][i]][idx[1][i]]
    actual_rating = arr[idx[0][i]][idx[1][i]]
    err = predicted_rating - actual_rating
    error.append(err)
  mse = (np.array(error) ** 2).mean()
  print('Validation MSE: ', mse)
  

In [94]:
validation_check()

Validation MSE:  13.206327761458075


In [96]:
### Creating new validation arrays from original data randomly again
for i in range(10):
  new_arr, idx = replace(arr, 0.1)
  validation_check()

Validation MSE:  13.23793563989452
Validation MSE:  13.325165122837326
Validation MSE:  13.207294623008817
Validation MSE:  13.180494257278031
Validation MSE:  13.179841998969904
Validation MSE:  13.244275441806645
Validation MSE:  13.129248377382421
Validation MSE:  13.313002542228993
Validation MSE:  13.26293492570781
Validation MSE:  13.271015821681216


# Results

## LF: 100, Iterations: 100
Each trial makes use of a different validation dataset

Training Error = 12.86
1. Validation Error = 13.206
2. Validation Error = 13.237
3. Validation Error = 13.207
4. Validation Error = 13.180
5. Validation Error = 13.179
6. Validation Error = 13.244
7. Validation Error = 13.129
8. Validation Error = 13.313
9. Validation Error = 13.262
10. Validation Error = 13.271
11. Validation Error = 13.325

Average Validation Error = 13.232







In [0]:
prediction = latent_user_preferences.dot(latent_item_features.T)

In [101]:
### Prediction values on entire Test Dataset

values = [zip(df.values[i], prediction[i]) for i in range(prediction.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -1.1972836166847933)","(8.79, 1.5647321351589467)","(-9.66, -7.2394244162386086)","(-8.16, -9.988349546981558)","(-7.52, -7.036595216662498)","(-8.5, -4.5323081953770625)","(-9.85, -4.051907719651013)","(4.17, -3.3591222463546773)","(-8.98, -8.302860786536945)","(-4.76, -3.5160275762218074)","(-8.5, -2.716364662135072)","(-6.75, -0.6716715468975085)","(-7.18, -5.134347338218933)","(8.45, 0.11908725361873959)","(-7.18, -4.777367182642235)","(-7.52, -9.83425681685249)","(-7.43, -5.821247611043145)","(-9.81, -5.484526165832135)","(-9.85, -6.504568440079951)","(-9.85, -5.924783049871823)","(-9.37, -1.2280856116134098)","(1.5, -4.577997060361018)","(-4.37, -3.792107383868851)","(-9.81, -9.702386628606131)","(-8.5, -4.193641759465168)","(1.12, -0.8547205423684991)","(7.82, 1.2554406160676124)","(2.86, -0.9521901250964012)","(9.13, 6.277098453751714)","(-7.43, -9.919056185311547)","(2.14, -0.9611348382656789)","(-4.08, 2.297112622503382)","(-9.08, -9.496976284698832)","(7.82, -1.6992778679929348)","(5.05, 4.210167613916918)","(4.95, 2.544127236399348)","(-9.17, -6.188792903807056)","(-8.4, -2.392103412876502)","(-8.4, -3.738662052211777)","(-8.4, -3.172700583153022)",...,"(8.59, -1.1304376966695293)","(3.59, 1.6057326653985426)","(-6.84, -1.3919255625692366)","(-9.03, -7.29502833585977)","(2.82, 0.7122264161843758)","(-1.36, 0.3793466516423297)","(-9.08, -8.553954825207155)","(8.3, 1.033707176019429)","(5.68, 1.9618285094988248)","(-4.81, -4.540695737078305)","(99.0, -5.4890876988055615)","(99.0, -0.9482235251138296)","(99.0, -9.980518064415298)","(99.0, -10.98447904734456)","(99.0, -9.996728587952687)","(99.0, -2.678739562768387)","(99.0, -7.905183546636063)","(-9.42, -6.557947790478049)","(99.0, -10.107774279418225)","(99.0, -7.937232870549387)","(99.0, -1.2444712771769855)","(-7.72, -5.3315491429352155)","(99.0, -1.8129730147732595)","(99.0, -5.336044993301098)","(99.0, -7.642513482859694)","(99.0, -8.418519316599895)","(99.0, -2.4568911434991083)","(99.0, -2.238252976650659)","(99.0, 1.954528557643171)","(99.0, -4.947525067555149)","(2.82, -2.6718863286388523)","(99.0, -5.1277296971078465)","(99.0, -0.7076041136357926)","(99.0, -6.273492199357841)","(99.0, -2.276858470380467)","(99.0, -3.7310553522726404)","(-5.63, -2.3506269547533933)","(99.0, -2.798906275799804)","(99.0, -8.255178637992765)","(99.0, -5.675803996306302)"
1,"(4.08, 3.63277210446559)","(-0.29, 2.162516548207998)","(6.36, 8.067568652898995)","(4.37, 5.71434918005848)","(-2.38, 3.268523176497036)","(-9.66, 1.671954236217768)","(-0.73, -2.0558131163295097)","(-5.34, 0.7632210267531646)","(8.88, 4.823172231966906)","(9.22, 9.176078010886048)","(6.75, 5.641183728978859)","(8.64, 0.6185454831632081)","(4.42, 1.5044469191214664)","(7.43, 0.5029838397906168)","(4.56, 1.7968051810017258)","(-0.97, 1.283910964237881)","(4.66, 1.0538920869998931)","(-0.68, -2.194961097277232)","(3.3, -1.2549859216987689)","(-1.21, -2.0513973189155847)","(0.87, 3.3198963671690307)","(8.64, 4.7704546488218025)","(8.35, 5.4378522331189805)","(9.17, 9.253797234559212)","(0.05, 2.153919092221421)","(7.57, 1.459195075191176)","(4.71, 2.3769583817012925)","(0.87, -2.378408263221073)","(-0.39, -0.3926163627606043)","(6.99, 4.675374385935252)","(6.5, 2.7937930911313527)","(-0.92, -2.205692776350599)","(7.14, 5.628318343276411)","(9.03, 3.5904848567998275)","(-1.8, -1.2824958936862956)","(0.73, 1.6850502766339672)","(7.09, 0.7459840294128016)","(3.4, 2.4666392402481563)","(-0.87, 2.388073250449343)","(7.91, 4.248107623578947)",...,"(-6.7, -1.1933503289516751)","(-3.35, -1.9793882924653758)","(-9.03, -2.1556706969557107)","(4.47, 5.755903012562944)","(4.08, 1.449570211999648)","(-3.83, -2.194818046332354)","(8.74, 6.058705591712222)","(1.12, -2.98037259773006

In [109]:
jokes_ratings = pd.DataFrame(prediction)
mean_idx = jokes_ratings.mean()
min_idx = mean_idx.idxmin()
max_idx = mean_idx.idxmax()
print('Worst Joke ID predicted: ', min_idx, '; Average Rating: ', mean_idx[min_idx])
print('Best Joke ID predicted: ', max_idx, '; Average Rating: ', mean_idx[max_idx])

Worst Joke ID predicted:  57 ; Average Rating:  -4.004488290910803
Best Joke ID predicted:  49 ; Average Rating:  3.5214865773076363
